## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://private-user-images.githubusercontent.com/181020547/465824799-12a2371b-8be2-4219-9b48-90503eb43c69.png?jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3NjAwNDgyMzcsIm5iZiI6MTc2MDA0NzkzNywicGF0aCI6Ii8xODEwMjA1NDcvNDY1ODI0Nzk5LTEyYTIzNzFiLThiZTItNDIxOS05YjQ4LTkwNTAzZWI0M2M2OS5wbmc_WC1BbXotQWxnb3JpdGhtPUFXUzQtSE1BQy1TSEEyNTYmWC1BbXotQ3JlZGVudGlhbD1BS0lBVkNPRFlMU0E1M1BRSzRaQSUyRjIwMjUxMDA5JTJGdXMtZWFzdC0xJTJGczMlMkZhd3M0X3JlcXVlc3QmWC1BbXotRGF0ZT0yMDI1MTAwOVQyMjEyMTdaJlgtQW16LUV4cGlyZXM9MzAwJlgtQW16LVNpZ25hdHVyZT1iYTRmYTAzYjkzYjA2MGE4ZTZlYjQ4ODU1OWIwY2VlZWU0Mzk0YzdmMjQ1YTlhMDMyNmI3NWNlZTQxNDdlZGViJlgtQW16LVNpZ25lZEhlYWRlcnM9aG9zdCJ9.a8477QD1J4Lrmys7jB8gt_H5pdiKBsKsu3npEqZjEpo)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

## Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [1]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

## Task 1: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [2]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

#### ❓ Question 1:

 Explain the interrelationships between the three states.  Why don't we just make a single huge state?

## Task 2: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [3]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

### ❓ Question 2:  

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

## Task 3: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [4]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 4: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [5]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## Task 5: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [6]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [7]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [8]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [9]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [10]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [11]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [12]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [13]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 6: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [14]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 7: Running the Deep Researcher

Now let's see the system in action! We'll use it to analyze a PDF document about how people use AI.

### Setup

We need to:
1. Load the PDF document
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [15]:
# Load the PDF document
from pathlib import Path
import PyPDF2

def load_pdf(pdf_path: str) -> str:
    """Load and extract text from PDF."""
    pdf_text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            pdf_text += page.extract_text() + "\n\n"
    return pdf_text

# Load the PDF about how people use AI
pdf_path = "data/howpeopleuseai.pdf"
pdf_content = load_pdf(pdf_path)

print(f"Loaded PDF with {len(pdf_content)} characters")
print(f"First 500 characters:\n{pdf_content[:500]}...")

Loaded PDF with 112460 characters
First 500 characters:
NBER WORKING PAPER SERIES
HOW PEOPLE USE CHATGPT
Aaron Chatterji
Thomas Cunningham
David J. Deming
Zoe Hitzig
Christopher Ong
Carl Yan Shan
Kevin Wadman
Working Paper 34255
http://www.nber.org/papers/w34255
NATIONAL BUREAU OF ECONOMIC RESEARCH
1050 Massachusetts Avenue
Cambridge, MA 02138
September 2025
We acknowledge help and comments from Joshua Achiam, Hemanth Asirvatham, Ryan 
Beiermeister,  Rachel Brown, Cassandra Duchan Solis, Jason Kwon, Elliott Mokski, Kevin Rao, 
Harrison Satcher,  Gawe...


In [16]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (3 concurrent researchers)
- **Clarification enabled** (will ask if research scope is unclear)

In [ ]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researchers
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 3")
print(f"  - Max Iterations: 4")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 3
  - Max Iterations: 4
  - Search API: Tavily


### Execute the Research

Now let's run the research! We'll ask the system to analyze the PDF and provide insights about how people use AI.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Multiple researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [18]:
# Create our research request with PDF context
research_request = f"""
I have a PDF document about how people use AI. Please analyze this document and provide insights about:

1. What are the main findings about how people are using AI?
2. What are the most common use cases?
3. What trends or patterns emerge from the data?

Here's the PDF content:

{pdf_content[:10000]}  # First 10k chars to stay within limits

...[content truncated for context window]
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with analyzing the PDF document about ChatGPT usage. You've provided a comprehensive research paper from the National Bureau of Economic Research titled "How People Use ChatGPT" and requested insights on: (1) main findings about how people use AI, (2) most common use cases, and (3) trends/patterns from the data. The document contains detailed usage statistics, classifications, and analysis from May 2024 to June 2025. I will now analyze this document and provide the requested insights based on the research findings presented.

Node: write_research_brief

Research Brief Generated:
I have a comprehensive NBER working paper titled "How People Use ChatGPT" (Working Paper No. 34255, September 2025) by Aaron Chatterji and colleagues that analyzes ChatGPT usage patterns from November 2022 through July 2025. I need you to thoroughly analyze this document and extract detailed insights on three speci

# Comprehensive Analysis of ChatGPT Usage Patterns: NBER Working Paper No. 34255

## Executive Summary

The NBER Working Paper "How People Use ChatGPT" by Aaron Chatterji and colleagues represents the most comprehensive analysis to date of how people interact with AI chatbots. The study tracked ChatGPT usage from its November 2022 launch through July 2025, revealing unprecedented adoption rates, significant demographic shifts, and evolving usage patterns that challenge conventional assumptions about AI adoption and application.

## Main Findings About AI Usage

### Unprecedented Growth and Adoption Statistics

ChatGPT has achieved the fastest technology adoption in recorded history. By July 2025, the platform reached over 750 million weekly active users, representing nearly 10% of the world's adult population [1]. The growth trajectory has been extraordinary: ChatGPT reached 1 million users within just 5 days of launch (by December 5, 2022) and achieved 100 million weekly active users within one year [3].

The scale of daily engagement is staggering. By June 2025, users were sending more than 2.6 billion messages per day—equivalent to over 30,000 messages per second. This represents a dramatic increase from 451 million daily messages in June 2024 [7]. Weekly message volume reached 18 billion by July 2025, with weekly active users doubling every 7-8 months while message volume grew even faster at a 5.8x increase compared to 3.2x user growth [3].

### Demographic Revolution: The Great Gender Reversal

Perhaps the most striking finding concerns the complete transformation of ChatGPT's user demographics. Initially, over 80% of users had typically male names when the platform first launched. However, this gender composition underwent a dramatic reversal. By January 2024, 37% of users with identifiable names were feminine, and by July 2025, this figure reached 52%, indicating complete gender parity had been achieved [1][4].

This represents one of the fastest demographic transitions documented in technology adoption history. The shift from a predominantly male user base to gender parity occurred in less than three years, defying typical patterns where new technologies maintain male-skewed adoption for extended periods.

### Age Demographics and Generational Patterns

The platform demonstrates a pronounced younger user skew. Nearly half of all adult messages originate from users under 26 years old [4]. More specifically, 52.99% of users fall within the 18-34 age range, with more than 45% being under age 25 [2]. This generational concentration suggests that ChatGPT has become deeply integrated into the digital habits of younger demographics.

### Education and Socioeconomic Patterns

Educational attainment correlates strongly with work-related usage patterns. More than 46% of users with a college degree use ChatGPT for work compared to 37% of those without a college degree [4]. Work usage is consistently more prevalent among educated users in highly-paid professional occupations, with writing tasks primarily originating from management and business occupations.

### Global Democratization and Geographic Patterns

Contrary to typical technology adoption patterns that favor educated males from wealthy countries, ChatGPT adoption has shown rapid global democratization. By May 2025, adoption growth rates in the lowest income countries were over 4 times higher than in the highest income countries [1]. Countries at the 50th percentile of GDP per capita now demonstrate similar ChatGPT usage rates to those at the 90th percentile, despite vast income differences.

The geographic distribution shows the United States leading with 17.2% of traffic, followed by India at 8.27% and Brazil at 5.73% [5]. This pattern indicates that AI access has genuinely democratized across economic boundaries, with middle-income countries showing particularly strong adoption rates.

### User Engagement Evolution

All user cohorts have significantly increased their usage beginning in late 2024 and early 2025 [1]. Early adopters from Q1 2023 are now sending 40% more messages per day than two years earlier, while users who signed up in late 2024 are sending nearly twice as many messages as when they first joined. Users spend an average of 7 minutes and 4 seconds per session, with an average of 4.5 pages viewed per visit and a bounce rate of 30.59% [5].

## Most Common Use Cases and Conversation Taxonomy

### The Big Three: Primary Usage Categories

The research employed OpenAI's conversation classifier to identify usage patterns across millions of conversations. Three dominant categories account for nearly 80% of all ChatGPT usage [4]:

1. **Practical Guidance**: 29% of all conversations - This category encompasses tutoring and teaching, how-to advice across various topics, and creative ideation [6][8]
2. **Seeking Information**: 24% of all conversations - Includes searching for information about people, current events, products, and recipes, functioning as a close substitute for web search [6]
3. **Writing**: 24% of all conversations - Encompasses automated production of emails, documents, and communications, plus editing, critiquing, summarizing, and translating user-provided text [6]

These three categories collectively represent 77% of all conversations, demonstrating remarkable consistency in how people approach AI interaction [6].

### Work Versus Non-Work Usage Revolution

One of the most significant findings challenges assumptions about AI's primary value proposition. The data reveals a fundamental shift toward personal use over time. Non-work-related messages have grown dramatically from 53% in June 2024 to more than 70% of all usage by June 2025 [4][7]. This trend continues to accelerate, with about 73% of consumer ChatGPT use now being non-work related [7].

This shift represents a paradigm change in AI adoption. While most economic analysis has focused on AI's impact on workplace productivity, the research demonstrates that personal, non-work applications are driving the majority of usage and growth.

### Deep Dive into Writing Tasks

Writing dominates work-related usage, representing approximately 40-42% of all work messages [4]. However, the nature of writing tasks reveals sophisticated usage patterns. Two-thirds of all writing messages ask ChatGPT to modify existing user text through editing, critiquing, translating, or other enhancement activities, rather than creating entirely new content from scratch [10]. This suggests users are leveraging AI as a collaborative partner rather than a replacement for human creativity.

### Computer Programming: Debunking the Developer Myth

Contrary to popular assumptions, computer programming accounts for only 4.2% of consumer conversations [7]. This low percentage debunks the widespread belief that ChatGPT is primarily a coding tool. The finding suggests that while programming applications receive significant media attention, they represent a small fraction of actual consumer usage.

### Educational Applications and Learning

Education represents 10.2% of all messages through tutoring or teaching functions [6]. This substantial portion indicates that ChatGPT has become an important supplementary educational resource, providing personalized learning support across various subjects and skill levels.

### User Intent Classification: Asking, Doing, Expressing

The researchers introduced an innovative "Asking/Doing/Expressing" taxonomy to categorize user interactions [5]:

- **Asking**: 49% of messages - Users seeking advice, guidance, or information [1][7]
- **Doing**: 40% of messages - Users requesting task completion or performance [1][7]  
- **Expressing**: 11% of messages - Personal reflection and exploration without specific information requests [1][7]

For work-specific messages, "Doing" dominates at 56% due to immediate output needs [7]. Interestingly, "Asking" messages consistently receive higher quality ratings than other categories and have grown faster over time [6], suggesting that users find AI most valuable as an information and guidance resource.

### Gender Differences in Application

The study identified notable gender differences in usage patterns. Females are more likely to use ChatGPT for writing and practical guidance, while males tend to utilize it more for technical help and information seeking [4]. These patterns suggest that different demographic groups have discovered distinct value propositions within the same technology platform.

## Trends and Patterns from the Data

### Explosive Growth Trajectory

The temporal growth data reveals unprecedented scaling. Daily message volume increased nearly six-fold from 451 million to 2.63 billion between June 2024 and June 2025 [7]. Weekly active users doubled from 400 million in February 2025 to 800 million by later in 2025 [5]. The platform now receives 5.8 billion monthly visits, representing a 7.606% month-over-month increase [5].

### Behavioral Maturation Patterns

User engagement demonstrates consistent deepening over time as people discover new use cases and benefit from improved models [1]. All signup cohorts followed remarkably similar patterns: relatively flat usage through 2024, followed by substantial increases beginning in early 2025 [3]. This suggests increasing integration of ChatGPT into users' weekly and daily routines rather than episodic usage.

### The Information-Seeking Revolution

One of the most significant behavioral shifts involves the growth of information-seeking conversations, which now represent 24% of all interactions, up from 14% a year earlier [6]. Researchers note this "appears to be a very close substitute for web search," suggesting that "if a quarter of conversations are information-seeking, some queries that would have gone to search may go toward conversational tools" [6].

This trend indicates that ChatGPT is becoming a major destination for finding information online, potentially reshaping how people access and interact with information on the internet.

### Technical Support Usage Decline

Interestingly, technical help usage dropped significantly from 12% in July 2024 to 5% in July 2025 [4]. This decline could indicate either improved user proficiency with the platform or the development of better self-service capabilities within the interface.

### Economic Value and Consumer Surplus

The study provides quantitative estimates of ChatGPT's economic value. U.S. users would need to be paid roughly $98 to give up generative AI for a month, implying at least $97 billion in annual consumer surplus in 2024 that traditional GDP measures don't capture [7]. This substantial figure demonstrates the significant economic value being created outside traditional measurement frameworks.

### Decision Support as Core Value Proposition

The research reveals that ChatGPT provides economic value primarily through decision support, which proves especially important in knowledge-intensive jobs [4]. About 81% of work-related messages involve obtaining/interpreting information and making decisions/solving problems [10]. This suggests that users view ChatGPT more as a thinking partner than simply a task automator, with users "not just looking for labor automation. They are looking for clarity" [7].

### Global Accessibility and Digital Divide Implications

The dramatic growth in lower-income countries, with adoption rates exceeding those of wealthier nations by more than four times [6], suggests that AI technologies may help bridge digital divides rather than exacerbate them. This democratization pattern represents a departure from typical technology adoption curves and could have significant implications for global economic development and educational access.

### Privacy and Methodological Innovation

The study employed groundbreaking privacy protection measures, including automated PII scrubbing, Data Clean Rooms for demographic analysis, and ensuring no researchers directly accessed message content or personal information [3]. The research analyzed over 1 million conversations using automated classifiers without human review of message content [10], demonstrating that large-scale behavioral research can be conducted while maintaining strict privacy standards.

## Future Implications and Conclusions

The research demonstrates that ChatGPT usage patterns are evolving rapidly, with 70% of use being personal rather than professional, indicating that "consumer habits are changing broadly" [6]. The platform has achieved a scale and integration level that positions it as a significant factor in how people access information, complete tasks, and make decisions in their daily lives.

The findings challenge many assumptions about AI adoption and usage. Rather than being primarily a workplace productivity tool or coding assistant, ChatGPT has emerged as a versatile personal assistant that people use for decision support, creative guidance, and information discovery. The rapid achievement of gender parity and global democratization suggests that AI technologies may follow different adoption patterns than previous digital innovations.

The economic implications are substantial, with billions of dollars in consumer surplus being generated outside traditional economic measurement systems. As AI technologies continue to evolve and integrate into daily life, understanding these usage patterns becomes crucial for policymakers, businesses, and researchers seeking to maximize the benefits while addressing potential challenges of widespread AI adoption.

### Sources

[1] How People Use ChatGPT - by David Deming - Forked Lightning: https://forklightning.substack.com/p/how-people-use-chatgpt

[2] Number of ChatGPT Users (October 2025) - Exploding Topics: https://explodingtopics.com/blog/chatgpt-users

[3] How People Use ChatGPT - by David Deming - Forked Lightning: https://forklightning.substack.com/p/how-people-use-chatgpt

[4] Here's what's in OpenAI's largest study on ChatGPT users - Quartz: https://qz.com/openai-study-chatgpt-users

[5] Latest ChatGPT Users Stats 2025 (Growth & Usage Report): https://www.demandsage.com/chatgpt-statistics/

[6] ChatGPT Study: 1 In 4 Conversations Now Seek Information: https://www.searchenginejournal.com/chatgpt-study-1-in-4-conversations-now-seek-information/556104/

[7] What Is ChatGPT Used For In 2025, Proven NBER Insights: https://binaryverseai.com/what-is-chatgpt-used-for/

[8] NBER Study Reveals Staggering ChatGPT Usage: 700M Users, 2.5: https://www.linkedin.com/posts/marium-lodhi-00057561_nber-research-report-on-chat-gpt-usage-activity-7374075759822303232-byIT

[9] How People Use ChatGPT | NBER: https://www.nber.org/papers/w34255

[10] How People Use ChatGPT - National Bureau of Economic Research: https://www.nber.org/system/files/working_papers/w34255/w34255.pdf


Research workflow completed!


## Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided a PDF and specific questions, it likely proceeded without clarification.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` multiple times to delegate to parallel researchers
- Each delegation specified a focused research topic

### Phase 4: Parallel Research
Multiple researchers worked simultaneously:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive report with:
- Well-structured sections
- Inline citations
- Sources listed at the end
- Balanced coverage of all findings

#### 🏗️ Activity #1: Try Different Configurations

You can experiment with different settings to see how they affect the research.  You may select three or more of the following settings (or invent your own experiments) and describe the results.

### Increase Parallelism
```python
"max_concurrent_research_units": 10  # More researchers working simultaneously
```

### Deeper Research
```python
"max_researcher_iterations": 8   # Supervisor can delegate more times
"max_react_tool_calls": 15      # Each researcher can search more
```

### Use Anthropic Native Search
```python
"search_api": "anthropic"  # Use Claude's built-in web search
```

### Disable Clarification
```python
"allow_clarification": False  # Skip clarification phase
```

## Key Takeaways

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

## Next Steps

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

### Deploy
- Use LangGraph Cloud for production deployment
- Add proper error handling and logging
- Implement rate limiting and cost controls
- Monitor research quality and costs